In [21]:
import numpy as np
import re
import pickle
import os
import seaborn as sns
import string
import pandas as pd
import tensorflow as tf
 

## Import Data

In [22]:
with open(r"C:\Users\Owner\Downloads\parallel\parallel\parallel-n\IITB.en-hi.en", 'r', encoding='utf-8') as f:
     english_sentences = f.read().split('\n')

In [23]:
with open(r"C:\Users\Owner\Downloads\parallel\parallel\parallel-n\IITB.en-hi.hi", 'r', encoding='utf-8') as f:
    hindi_sentences = f.read().split('\n')

In [24]:
english_sentences

['Give your application an accessibility workout',
 'Accerciser Accessibility Explorer',
 'The default plugin layout for the bottom panel',
 'The default plugin layout for the top panel',
 'A list of plugins that are disabled by default',
 'Highlight duration',
 'The duration of the highlight box when selecting accessible nodes',
 'Highlight border color',
 'The color and opacity of the highlight border.',
 'Highlight fill color',
 'The color and opacity of the highlight fill.',
 'API Browser',
 'Browse the various methods of the current accessible',
 'Hide private attributes',
 'Method',
 'Property',
 'Value',
 'IPython Console',
 'Interactive console for manipulating currently selected accessible',
 'Event monitor',
 '_ Monitor Events',
 'C _ lear Selection',
 'Everything',
 'Selected application',
 'Selected accessible',
 'Source',
 'Event Monitor',
 'Shows events as they occur from selected types and sources',
 'Highlight last event entry',
 'Start / stop event recording',
 'Clear 

In [25]:
hindi_sentences


['अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें',
 'एक्सेर्साइसर पहुंचनीयता अन्वेषक',
 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका',
 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका',
 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है',
 'अवधि को हाइलाइट रकें',
 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि',
 'सीमांत (बोर्डर) के रंग को हाइलाइट करें',
 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। ',
 'भराई के रंग को हाइलाइट करें',
 'हाइलाइट किया गया भराई का रंग और पारदर्शिता। ',
 'एपीआई विचरक',
 'इस समय जिसे प्राप्त किया गया हो, उसकी विभिन्न विधियों (मेथड) में विचरण करें',
 'निजी गुणों को छिपाएं',
 'विधि',
 'गुणधर्म',
 'मान',
 'आईपाइथन कन्सोल',
 'इस समय चुने गए एक्सेसेबेल से काम लेने के लिए अंतर्क्रियात्मक कन्सोल',
 'घटना मानिटर',
 'घटनाओं को मानिटर करें (_ M) ',
 'चुनाव को हटाएं (C _) ',
 'सभी',
 'चुने गए अनुप्रयोग',
 'चुने गए एक्सेसेबेल',
 'स्रोत',
 'घटना मानिटर',
 'चुने गए प्रकारों और स्रोतों से घटनाएं जैसे-जैसे घटित होती हैं, उन्हें दर्शाता है',
 'अंतिम प्रविष्ट घटन

In [26]:
type(hindi_sentences)

list

## Preprocess Data

In [27]:
#Some parameters
vocab_size = 10000
total_sentences = 50000
maxlen = 16
epochs = 70
validation_split = 0.05
max_sentence_length= maxlen

In [28]:
def preprocess(text):
    text = ''.join(ch for ch in text if ch not in string.punctuation)
    text = text.lower()
    text = re.sub(r'\d','',text)
    text = re.sub(r'\s+',' ',text)
    text = text.strip()
    return text

In [29]:
eng_sentence = [preprocess(en) for en in english_sentences]

In [30]:
eng_sentence

['give your application an accessibility workout',
 'accerciser accessibility explorer',
 'the default plugin layout for the bottom panel',
 'the default plugin layout for the top panel',
 'a list of plugins that are disabled by default',
 'highlight duration',
 'the duration of the highlight box when selecting accessible nodes',
 'highlight border color',
 'the color and opacity of the highlight border',
 'highlight fill color',
 'the color and opacity of the highlight fill',
 'api browser',
 'browse the various methods of the current accessible',
 'hide private attributes',
 'method',
 'property',
 'value',
 'ipython console',
 'interactive console for manipulating currently selected accessible',
 'event monitor',
 'monitor events',
 'c lear selection',
 'everything',
 'selected application',
 'selected accessible',
 'source',
 'event monitor',
 'shows events as they occur from selected types and sources',
 'highlight last event entry',
 'start stop event recording',
 'clear event lo

In [31]:
hindi_sentence = [re.sub('[a-zA-Z]','',preprocess(hi))for hi in hindi_sentences]

In [32]:
hindi_sentence

['अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें',
 'एक्सेर्साइसर पहुंचनीयता अन्वेषक',
 'निचले पटल के लिए डिफोल्ट प्लगइन खाका',
 'ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका',
 'उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है',
 'अवधि को हाइलाइट रकें',
 'पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से की अवधि',
 'सीमांत बोर्डर के रंग को हाइलाइट करें',
 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता।',
 'भराई के रंग को हाइलाइट करें',
 'हाइलाइट किया गया भराई का रंग और पारदर्शिता।',
 'एपीआई विचरक',
 'इस समय जिसे प्राप्त किया गया हो उसकी विभिन्न विधियों मेथड में विचरण करें',
 'निजी गुणों को छिपाएं',
 'विधि',
 'गुणधर्म',
 'मान',
 'आईपाइथन कन्सोल',
 'इस समय चुने गए एक्सेसेबेल से काम लेने के लिए अंतर्क्रियात्मक कन्सोल',
 'घटना मानिटर',
 'घटनाओं को मानिटर करें ',
 'चुनाव को हटाएं ',
 'सभी',
 'चुने गए अनुप्रयोग',
 'चुने गए एक्सेसेबेल',
 'स्रोत',
 'घटना मानिटर',
 'चुने गए प्रकारों और स्रोतों से घटनाएं जैसेजैसे घटित होती हैं उन्हें दर्शाता है',
 'अंतिम प्रविष्ट घटना को हाइलाइट करो',
 'घटना 

In [33]:
 len(eng_sentence),len(hindi_sentence)

(1659084, 1659084)

In [34]:
#Remove duplicate sentences
english_unique = set()
eng_sentence_temp = []
hindi_sentence_temp = []
l = len(english_sentences)
for i in range(l):
    if english_sentences[i] not in english_unique:
        english_unique.add(eng_sentence[i])
        eng_sentence_temp.append(eng_sentence[i])
        hindi_sentence_temp.append(hindi_sentence[i])

eng_sentence = eng_sentence_temp
hindi_sentence = hindi_sentence_temp
  

## Load Data

In [35]:
en_data = []
hi_data = []

cnt = 0

for (en,hi) in zip(eng_sentence, hindi_sentence):
    l = min(len(en.split()), len(hi.split()))
    if l <= maxlen:
        en_data.append(en)
        hi_data.append(hi)
    cnt += 1
    if cnt == total_sentences:
        break

In [40]:
len(en_data), print(len(hi_data))

49249


(49249, None)

In [41]:
hi_data = ['<START> ' + hi + ' <END>' for hi in hi_data]

In [42]:
hi_data

['<START> अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें <END>',
 '<START> एक्सेर्साइसर पहुंचनीयता अन्वेषक <END>',
 '<START> निचले पटल के लिए डिफोल्ट प्लगइन खाका <END>',
 '<START> ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका <END>',
 '<START> उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है <END>',
 '<START> अवधि को हाइलाइट रकें <END>',
 '<START> पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से की अवधि <END>',
 '<START> सीमांत बोर्डर के रंग को हाइलाइट करें <END>',
 '<START> हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। <END>',
 '<START> भराई के रंग को हाइलाइट करें <END>',
 '<START> हाइलाइट किया गया भराई का रंग और पारदर्शिता। <END>',
 '<START> एपीआई विचरक <END>',
 '<START> इस समय जिसे प्राप्त किया गया हो उसकी विभिन्न विधियों मेथड में विचरण करें <END>',
 '<START> निजी गुणों को छिपाएं <END>',
 '<START> विधि <END>',
 '<START> गुणधर्म <END>',
 '<START> मान <END>',
 '<START> आईपाइथन कन्सोल <END>',
 '<START> इस समय चुने गए एक्सेसेबेल से काम लेने के लिए अंतर्क्रियात्मक कन्सोल <END>',
 '<ST

## Tokenize Text

In [47]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize English texts
en_tokenizer = Tokenizer(filters='', oov_token='<OOV>', lower=False)
en_tokenizer.fit_on_texts(en_data)
en_sequences = en_tokenizer.texts_to_sequences(en_data)

# Tokenize Hindi texts
hi_tokenizer = Tokenizer(filters='', oov_token='<OOV>', lower=False)
hi_tokenizer.fit_on_texts(hi_data)
hi_sequences = hi_tokenizer.texts_to_sequences(hi_data)

 

# Calculate vocabulary sizes
english_vocab_size = len(en_tokenizer.word_index) + 1
hindi_vocab_size = len(hi_tokenizer.word_index) + 1

# Print vocabulary sizes
print("English Vocab Size: ", english_vocab_size)
print("Hindi Vocab Size: ", hindi_vocab_size)

English Vocab Size:  3622
Hindi Vocab Size:  3835


In [48]:
en_tokenizer.word_index

{'<OOV>': 1,
 'the': 2,
 'to': 3,
 'a': 4,
 'of': 5,
 's': 6,
 'file': 7,
 'project': 8,
 'for': 9,
 'in': 10,
 'not': 11,
 'be': 12,
 'disc': 13,
 'is': 14,
 'files': 15,
 'you': 16,
 'and': 17,
 'or': 18,
 'name': 19,
 'add': 20,
 'this': 21,
 'new': 22,
 'on': 23,
 'please': 24,
 'plugin': 25,
 'with': 26,
 'image': 27,
 'no': 28,
 'an': 29,
 'select': 30,
 'video': 31,
 'from': 32,
 'remove': 33,
 'use': 34,
 'selected': 35,
 'all': 36,
 'directory': 37,
 'current': 38,
 'show': 39,
 'move': 40,
 'whether': 41,
 'error': 42,
 'do': 43,
 'dvd': 44,
 'could': 45,
 'open': 46,
 'text': 47,
 'cd': 48,
 'want': 49,
 'are': 50,
 'source': 51,
 'it': 52,
 'c': 53,
 'audio': 54,
 'message': 55,
 'that': 56,
 'will': 57,
 'path': 58,
 'save': 59,
 'program': 60,
 'window': 61,
 'line': 62,
 'create': 63,
 'set': 64,
 'used': 65,
 'actor': 66,
 'data': 67,
 'burning': 68,
 'onto': 69,
 'copy': 70,
 'start': 71,
 'anjuta': 72,
 'target': 73,
 'type': 74,
 'next': 75,
 'options': 76,
 'if': 77

In [49]:
hi_tokenizer.word_index

{'<OOV>': 1,
 '<START>': 2,
 '<END>': 3,
 'के': 4,
 'को': 5,
 'है': 6,
 'करें': 7,
 'नहीं': 8,
 'में': 9,
 'का': 10,
 'लिए': 11,
 'से': 12,
 'फ़ाइल': 13,
 'की': 14,
 'पर': 15,
 'डिस्क': 16,
 'रहा': 17,
 'एक': 18,
 'परियोजना': 19,
 'और': 20,
 'नाम': 21,
 'क्या': 22,
 'या': 23,
 'फाइल': 24,
 'छवि': 25,
 'जोड़ें': 26,
 'कोई': 27,
 'कर': 28,
 'कृपया': 29,
 'इंच': 30,
 'यह': 31,
 'हैं': 32,
 'किया': 33,
 'नया': 34,
 'सभी': 35,
 'चुनें': 36,
 'साथ': 37,
 'करने': 38,
 'वीडियो': 39,
 'जा': 40,
 'प्लगइन': 41,
 'चयनित': 42,
 'आप': 43,
 'जाएँ': 44,
 'गया': 45,
 'प्रयोग': 46,
 'त्रुटि': 47,
 'निर्देशिका': 48,
 'खोलें': 49,
 'स्थिति': 50,
 'बदलें': 51,
 'दिखाएँ': 52,
 'पाठ': 53,
 'विंडो': 54,
 'ले': 55,
 'स्थान': 56,
 'संदेश': 57,
 'खाली': 58,
 'योग्य': 59,
 'लक्ष्य': 60,
 'कर्ता': 61,
 'प्रोजेक्ट': 62,
 'रखें': 63,
 'इस': 64,
 'प्रोग्राम': 65,
 'उपयोग': 66,
 'हटाएँ': 67,
 'सेट': 68,
 'पंक्ति': 69,
 'सहेजें': 70,
 'सूची': 71,
 'डीवीडी': 72,
 'विकल्प': 73,
 'करना': 74,
 'आकार': 75,
 'प्लगिन': 76,
 '

## Prepare input Output Data For English to Hindi Translation Model

In [50]:
#Prepare encoder data
encoder_inputs = tf.keras.preprocessing.sequence.pad_sequences(en_sequences, maxlen=maxlen, padding='post')

In [51]:
encoder_inputs

array([[1127,   79,  200, ...,    0,    0,    0],
       [ 984,  590, 1450, ...,    0,    0,    0],
       [   2,  153,   25, ...,    0,    0,    0],
       ...,
       [ 685,    0,    0, ...,    0,    0,    0],
       [ 181,    0,    0, ...,    0,    0,    0],
       [  28, 1190,  281, ...,    0,    0,    0]])

In [53]:
#Prepare decoder data
decoder_inputs = []
decoder_outputs = []

for hi in hi_sequences:
    decoder_inputs.append(hi[:-1])
    decoder_outputs.append(hi[1:])

decoder_inputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_inputs, maxlen=maxlen, padding='post')
decoder_outputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_outputs, maxlen=maxlen, padding='post')

# Training and Testing split
# 95%, 5%
split = int(0.9 * total_sentences)

X_train = [encoder_inputs[:split], decoder_inputs[:split]]
y_train = decoder_outputs[:split]

# Test data to evaluate our NMT model using BLEU score
X_test = en_data[:split]
y_test = hi_data[:split]

print(X_train[0].shape, X_train[1].shape, y_train.shape)

(45000, 16) (45000, 16) (45000, 16)


## Initialize model

In [59]:
import numpy as np
import tensorflow as tf

class LSTMModel(tf.keras.Model):
    def __init__(self, encoder_vocab_size=None, decoder_vocab_size=None, embedding_size=128, num_rnn_units=32, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.encoder_vocab_size = encoder_vocab_size
        self.decoder_vocab_size = decoder_vocab_size
        self.embedding_size = embedding_size
        # encoder
        self.input_1 = tf.keras.layers.InputLayer(input_shape=(None,), name='input_1')
        self.embedding_1 = tf.keras.layers.Embedding(encoder_vocab_size, embedding_size, mask_zero=True, name='embedding_1')
        self.encoder_lstm = tf.keras.layers.LSTM(num_rnn_units, return_state=True, name='encoder_lstm')
        # decoder
        self.input_2 = tf.keras.layers.InputLayer(input_shape=(None,), name='input_2')
        self.embedding_2 = tf.keras.layers.Embedding(decoder_vocab_size, embedding_size, mask_zero=True, name='embedding_2')
        self.decoder_lstm = tf.keras.layers.LSTM(num_rnn_units, activation='relu', return_sequences=True, return_state=True, name='decoder_lstm')
        self.token_layer = tf.keras.layers.Dense(decoder_vocab_size, activation='softmax', name='token_layer')

    def call(self, inputs):
        encoder_input = self.input_1(inputs[0])
        decoder_input = self.input_2(inputs[1])
        # encode the inputs
        encoder_embed = self.embedding_1(encoder_input)
        # run rnn on the encoded sequence
        _, state_h, state_c = self.encoder_lstm(encoder_embed)
        # decode the target
        decoder_embed = self.embedding_2(decoder_input)
        x, _, _ = self.decoder_lstm(decoder_embed, initial_state=[state_h, state_c])
        return self.token_layer(x)

    def predict_sequence(self, text, input_tokenizer, output_tokenizer, max_len):
        if type(text) != list:
            text = [text]
        input_sequence = input_tokenizer.texts_to_sequences(text)
        if type(input_sequence) == list:
            input_sequence = np.array(input_sequence)
        encoder_embed = self.embedding_1(input_sequence)
        # run rnn on the encoded sequence
        _, next_h, next_c = self.encoder_lstm(encoder_embed)
        curr_token = [[output_tokenizer.word_index['<START>']]]

        out_seq = ""
        for _ in range(max_len):
            decoder_embedding = self.embedding_2(np.array(curr_token))
            x, next_h, next_c = self.decoder_lstm(decoder_embedding, initial_state=[next_h, next_c])
            x = self.token_layer(x)
            next_token = np.argmax(x[0, 0, :])
            next_word = output_tokenizer.index_word[next_token]
            if next_word == "<END>":
                break
            curr_token[0][0] = next_token
            out_seq += " " + next_word
        return out_seq.strip()

 

In [60]:
en_hi_model = LSTMModel(encoder_vocab_size=english_vocab_size, decoder_vocab_size= hindi_vocab_size ,embedding_size=256, num_rnn_units=64) 
en_hi_model(np.array([[[1]*max_sentence_length], [[1]*max_sentence_length]]))
en_hi_model.summary()
loss = tf.keras.losses.SparseCategoricalCrossentropy()

Model: "lstm_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     multiple                  927232    
                                                                 
 encoder_lstm (LSTM)         multiple                  82176     
                                                                 
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     multiple                  981760    
                                                                 
 decoder_lstm (LSTM)         multiple                  82176     
                                                                 
 token_layer (Dense)         multiple                  2

In [63]:
#Save model after each epoch
save_model_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=r"F:\Guided Project\RTA\New\model5",
    monitor='val_accuracy',
    mode='max'
)
en_hi_model.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])
en_hi_model.fit(X_train, y_train, epochs=10,batch_size= 64, validation_split=validation_split, callbacks=[save_model_callback, tf.keras.callbacks.TerminateOnNaN()])

Epoch 1/10
668/668 [==============================] - ETA: 0s - loss: 2.0798 - accuracy: 0.5936

INFO:tensorflow:Assets written to: F:\Guided Project\RTA\New\model5\assets


INFO:tensorflow:Assets written to: F:\Guided Project\RTA\New\model5\assets


668/668 [==============================] - 102s 150ms/step - loss: 2.0798 - accuracy: 0.5936 - val_loss: 6.0398 - val_accuracy: 0.2789
Epoch 2/10
668/668 [==============================] - ETA: 0s - loss: 1.6215 - accuracy: 0.6760

INFO:tensorflow:Assets written to: F:\Guided Project\RTA\New\model5\assets


...

INFO:tensorflow:Assets written to: F:\Guided Project\RTA\New\model5\assets


668/668 [==============================] - 114s 170ms/step - loss: 0.5238 - accuracy: 0.9044 - val_loss: 7.0536 - val_accuracy: 0.2642
Epoch 10/10
668/668 [==============================] - ETA: 0s - loss: 0.4752 - accuracy: 0.9130

INFO:tensorflow:Assets written to: F:\Guided Project\RTA\New\model5\assets


INFO:tensorflow:Assets written to: F:\Guided Project\RTA\New\model5\assets


668/668 [==============================] - 124s 184ms/step - loss: 0.4752 - accuracy: 0.9130 - val_loss: 7.0627 - val_accuracy: 0.2772


In [64]:
en_hi_model.save_weights(r"F:\Guided Project\RTA\New\model5\en_hi_model.h5")

In [68]:
# Testing and Analysis

candidates = []
references = []

ctr = 20 
i = 0

while ctr > 0:
    l = len(X_test[i].split())
    if l <= maxlen:  # Choose only sentences of length in range [5,15]
        pred_sentence = en_hi_model.predict_sequence(X_test[i], en_tokenizer, hi_tokenizer, max_len=maxlen)
        candidates.append(pred_sentence.split())

        print("Input: ", X_test[i])
        print("Prediction: ", pred_sentence)

       
        
        print("Dataset Reference: ", ' '.join(y_test[i].split()[1:-1]))
        print()
        references.append([y_test[i].split()[1:-1]])

        ctr -= 1
    i += 1

 


Input:  give your application an accessibility workout
Prediction:  अपने अनुप्रयोग को पहुंचनीयता संकेत नियंत्रित नहीं
Dataset Reference:  अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें

Input:  accerciser accessibility explorer
Prediction:  एक्सेर्साइसर पहुंचनीयता अन्वेषक
Dataset Reference:  एक्सेर्साइसर पहुंचनीयता अन्वेषक

Input:  the default plugin layout for the bottom panel
Prediction:  ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका
Dataset Reference:  निचले पटल के लिए डिफोल्ट प्लगइन खाका

Input:  the default plugin layout for the top panel
Prediction:  ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका
Dataset Reference:  ऊपरी पटल के लिए डिफोल्ट प्लगइन खाका

Input:  a list of plugins that are disabled by default
Prediction:  उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
Dataset Reference:  उन प्लगइनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है

Input:  highlight duration
Prediction:  हाइलाइट अवधिः
Dataset Reference:  अवधि को हाइलाइट रकें

Input:  the duration of the highlight 